<a href="https://colab.research.google.com/github/inderpreetsingh01/ml_machine_coding/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class MultinomialNaiveBayes:
    def __init__(self, alpha=1.0):  # Laplace smoothing
        self.alpha = alpha
        self.class_log_prior_ = None
        self.feature_log_prob_ = None
        self.classes_ = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        class_count = np.zeros(n_classes)
        feature_count = np.zeros((n_classes, n_features))

        for idx, c in enumerate(self.classes_):
            X_c = X[y == c]
            class_count[idx] = X_c.shape[0]
            feature_count[idx, :] = np.sum(X_c, axis=0)

        # Log priors: log(P(y))
        self.class_log_prior_ = np.log(class_count / n_samples)

        # Log likelihoods: log(P(x_j | y))
        smoothed_fc = feature_count + self.alpha
        smoothed_totals = np.sum(smoothed_fc, axis=1, keepdims=True)
        self.feature_log_prob_ = np.log(smoothed_fc / smoothed_totals)

    def predict_log_proba(self, X):
        return X @ self.feature_log_prob_.T + self.class_log_prior_

    def predict(self, X):
        log_probs = self.predict_log_proba(X)
        return self.classes_[np.argmax(log_probs, axis=1)]

In [ ]:
# 6 samples, 4 features (word counts), 2 classes (0 and 1)
X = np.array([
    [3, 0, 1, 0],
    [2, 0, 1, 1],
    [0, 1, 0, 3],
    [0, 1, 0, 2],
    [1, 0, 2, 0],
    [0, 2, 0, 1]
])
y = np.array([0, 0, 1, 1, 0, 1])

model = MultinomialNaiveBayes(alpha=1.0)
model.fit(X, y)
preds = model.predict(X)

print("Predictions:", preds)

In [ ]:
class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]

        self.mean_ = np.zeros((n_classes, n_features))
        self.var_ = np.zeros((n_classes, n_features))
        self.priors_ = np.zeros(n_classes)

        for idx, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.mean_[idx, :] = np.mean(X_c, axis=0)
            self.var_[idx, :] = np.var(X_c, axis=0) + 1e-9  # avoid divide by 0
            self.priors_[idx] = X_c.shape[0] / X.shape[0]

    def _gaussian_log_likelihood(self, class_idx, x):
        mean = self.mean_[class_idx]
        var = self.var_[class_idx]
        return -0.5 * np.sum(np.log(2 * np.pi * var) + ((x - mean) ** 2) / var)

    def predict(self, X):
        preds = []
        for x in X:
            posteriors = []
            for i in range(len(self.classes_)):
                prior = np.log(self.priors_[i])
                likelihood = self._gaussian_log_likelihood(i, x)
                posterior = prior + likelihood
                posteriors.append(posterior)
            preds.append(self.classes_[np.argmax(posteriors)])
        return np.array(preds)

In [1]:
import numpy as np

class GaussianNaiveBayes:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing
        self.classes_ = None
        self.class_priors_ = {}
        self.class_feature_means_ = {}
        self.class_feature_vars_ = {}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        for cls in self.classes_:
            X_c = X[y == cls]
            self.class_priors_[cls] = X_c.shape[0] / X.shape[0]
            self.class_feature_means_[cls] = np.mean(X_c, axis=0)
            self.class_feature_vars_[cls] = np.var(X_c, axis=0) + self.var_smoothing

    def _gaussian_likelihood(self, cls, x):
        mean = self.class_feature_means_[cls]
        var = self.class_feature_vars_[cls]
        coeff = 1 / np.sqrt(2 * np.pi * var)
        exp_term = np.exp(-0.5 * ((x - mean) ** 2) / var)
        return coeff * exp_term

    def _log_posterior(self, x):
        log_probs = []
        for cls in self.classes_:
            log_prior = np.log(self.class_priors_[cls])
            log_likelihood = np.sum(np.log(self._gaussian_likelihood(cls, x)))
            log_probs.append(log_prior + log_likelihood)
        return np.array(log_probs)

    def predict_proba(self, X):
        """
        Returns probability distribution over classes for each sample.
        Uses softmax over log-posteriors for numerical stability.
        """
        proba = []
        for x in X:
            log_probs = self._log_posterior(x)
            log_probs -= np.max(log_probs)  # for stability
            probs = np.exp(log_probs)
            probs /= np.sum(probs)
            proba.append(probs)
        return np.array(proba)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

    def confusion_matrix(self, X, y):
        """
        Returns a confusion matrix of shape (n_classes, n_classes)
        """
        y_pred = self.predict(X)
        n_classes = len(self.classes_)
        cm = np.zeros((n_classes, n_classes), dtype=int)
        for true, pred in zip(y, y_pred):
            cm[int(true), int(pred)] += 1
        return cm

In [ ]:
 # Simulate data
np.random.seed(0)
X = np.random.randn(150, 4)
y = np.repeat([0, 1, 2], 50)
X[y == 1] += 2
X[y == 2] -= 2

model = GaussianNaiveBayes()
model.fit(X, y)

proba = model.predict_proba(X)
preds = model.predict(X)
acc = model.score(X, y)
cm = model.confusion_matrix(X, y)

print("Accuracy:", acc)
print("Confusion Matrix:\n", cm)